#4-5 アンサンブル学習

## 概要

**アンサンブル学習は、性能の低い学習器（弱学習機）を組み合わせて、性能の高い学習器を作る方法です。**

アンサンブル学習の代表的な手法として、「バギング」と「ブースティング」があります。


###バギング

「バギング」は、弱学習器を並列に学習して組み合わせる手法です。
元データから、重複を許してランダムにデータを取得（ブーストラップ）し、並列に学習して、多数決で分類結果を出力します。

<img src="https://raw.githubusercontent.com/t-date/DataScience/master/fig/04_05_01.jpg?raw=true" width="550px">

「バギング」の代表的な手法として、「ランダムフォレスト」があります。「ランダムフォレスト」は、弱学習器として決定木を使い、複数の決定木を使うことで精度向上を図ります。

###ブースティング

「ブースティング」は、弱学習器を順番に学習して組み合わせて強くしていく手法です。
前の学習器が誤分類したデータを優先的に正しく分類できるように学習していきます。

<img src="https://raw.githubusercontent.com/t-date/DataScience/master/fig/04_05_02.jpg?raw=true" width="550px">

「ブースティング」の代表的な手法として、「勾配ブースティング」があります。
勾配ブースティングでは、損失関数として、勾配降下法を使用しています。「勾配ブー
スティング」を高速に実行できるようにC++ で開発した「XGBoost」が有名です。

##バギングとペースティング

多様性の高い分類器を用意するための方法のひとつは、大きく異なる訓練アルゴリズムを使うことだが、すべての分類器で同じ訓練アルゴリズムを使いつつ、訓練セットから無作為に別々のサブセットをサンプリングして訓練するというアプローチがあります。サンプリングが重複ありで行われるときにはバギング（bagging、bootstrap
aggreating の略。）、重複なしで行われるときにはペースティングと呼びます。

言い換えると、バギングとペースティングはともに複数の予測器が同じ訓練インスタンスを複数回サンプリングすることを認めるが、同じ予測器が同じ訓練インスタンスを複数回サンプリングすることを認めるのはバギングだけであります。概要の図 は、このようなサンプリングと訓練のプロセスを描いたものであります。

すべての分類器を予測したら、アンサンブルは単純にすべての予測器の予測を集計して新インスタンスに対する予測をすることができます。集計関数は、一般に分類ではモード（statistical mode、
つまりハード投票分類器と同様に、予測の最頻値を取る）、回帰では平均である。個別の予測器は、訓練セット全体を対象として訓練したときよりもバイアスが高くなっているが、集計によってバイ
アスと分散の両方が下がります。一般に、もとの訓練セット全体でひとつの予測器を訓練したときと比べて、アンサンブルでは、バイアスは同じようなものだが、分散は下がっています。

異なるCPU コアや異なるサーバーを使ってすべての分類器を並列に訓
練できる。同様に、予測も並列に実行できます。バギングやペースティングの人気が高い理由のひとつがこのスケーラビリティの高さであります。

###scikit-learn におけるバギングとペースティング

scikit-learn は、バギングとペースティングの両方に対してBaggingClassifier クラスという単純なAPI を提供しています（回帰の場合は、BaggingRegressor クラス）。次のコードは、500 個の決定木分類器によるアンサンブルを訓練します。個々の分類器は、重複ありで訓練セットから100 個の訓練インスタンスを無作為抽出します。これはバギングの例だが、ペースティングを使いたい場合はbootstrap=False を設定すればよいです。n_jobs パラメータは、scikit-learn に訓
練、予測のために使うCPU コアの数を指示します（−1 にすると、scikit-learn は使えるすべてのコアを使う）。

In [0]:
# Python 2, 3 をサポートします
from __future__ import division, print_function, unicode_literals

# 標準ライブラリのインポート
import numpy as np
import os

# 乱数の固定
np.random.seed(42)

# プロットの設定
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# 保存先ディレクトリの設定
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "ensembles"
IMAGE_DIR = "images"

os.makedirs(os.path.join(PROJECT_ROOT_DIR, IMAGE_DIR, CHAPTER_ID), exist_ok=True)

def image_path(fig_id):
    return os.path.join(PROJECT_ROOT_DIR, IMAGE_DIR, CHAPTER_ID, fig_id)

def save_fig(fig_id, tight_layout=True):
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(image_path(fig_id) + ".png", format='png', dpi=300)

In [0]:
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_moons

X, y = make_moons(n_samples=500, noise=0.30, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [0]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

bag_clf = BaggingClassifier(
    DecisionTreeClassifier(random_state=42), n_estimators=500,
    max_samples=100, bootstrap=True, n_jobs=-1, random_state=42)
bag_clf.fit(X_train, y_train)
y_pred = bag_clf.predict(X_test)

In [0]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred))

In [0]:
tree_clf = DecisionTreeClassifier(random_state=42)
tree_clf.fit(X_train, y_train)
y_pred_tree = tree_clf.predict(X_test)
print(accuracy_score(y_test, y_pred_tree))

In [0]:
from matplotlib.colors import ListedColormap

def plot_decision_boundary(clf, X, y, axes=[-1.5, 2.5, -1, 1.5], alpha=0.5, contour=True):
    x1s = np.linspace(axes[0], axes[1], 100)
    x2s = np.linspace(axes[2], axes[3], 100)
    x1, x2 = np.meshgrid(x1s, x2s)
    X_new = np.c_[x1.ravel(), x2.ravel()]
    y_pred = clf.predict(X_new).reshape(x1.shape)
    custom_cmap = ListedColormap(['#fafab0','#9898ff','#a0faa0'])
    plt.contourf(x1, x2, y_pred, alpha=0.3, cmap=custom_cmap)
    if contour:
        custom_cmap2 = ListedColormap(['#7d7d58','#4c4c7f','#507d50'])
        plt.contour(x1, x2, y_pred, cmap=custom_cmap2, alpha=0.8)
    plt.plot(X[:, 0][y==0], X[:, 1][y==0], "yo", alpha=alpha)
    plt.plot(X[:, 0][y==1], X[:, 1][y==1], "bs", alpha=alpha)
    plt.axis(axes)
    plt.xlabel(r"$x_1$", fontsize=18)
    plt.ylabel(r"$x_2$", fontsize=18, rotation=0)

In [0]:
plt.figure(figsize=(11,4))
plt.subplot(121)
plot_decision_boundary(tree_clf, X, y)
plt.title("Decision Tree", fontsize=14)
plt.subplot(122)
plot_decision_boundary(bag_clf, X, y)
plt.title("Decision Trees with Bagging", fontsize=14)
save_fig("decision_tree_without_and_with_bagging_plot")
plt.show()

上図は、ひとつの決定木を使ったときの決定境界と500 個の木によるバギングアンサンブル（上記のコード）を使ったときの決定境界を比較したものであります。ご覧のようにアンサンブルの予
測は、単独の決定木による予測よりもはるかに汎化性能が高くなります。アンサンブルのバイアスはほぼ同じだが、分散は小さくなります（訓練セットでの予測誤りはほぼ同じだが、決定境界の不規則度は下がる）。

ブートストラップ法により、個々の予測器が訓練に使うサブセットの多様性が若干上がるため、バギングの方がペースティングよりもバイアスが少し高くなるが、それは予測器の相関が下がると
いうことであるため、アンサンブルの分散は下がります。全体として、バギングの方がペースティングより
もよいモデルになることが多くあります。一般に、バギングの方が好まれているのはそのためであります。しかし、時間とCPU パワーに余裕がある場合には、バギングとペースティングを交差検証して、性能
のよい方を選ぶことができます。

###ランダムフォレスト

ランダムフォレストは、決定木のアンサンブルで、一般にバギングメソッドで訓練され（ペースティングが使われる場合もある）、
max_samples は訓練セットサイズに設定されます。BaggingClassifier を構築し、それをDecisionTreeClassifier に渡さなくても、RandomForestClassifier クラスを使えるようになっています。

RandomForestClassifier クラスの方が便利なだけでなく、決定木に最適化されています（同様に、回帰のタスクのためにRandomForestRegressor クラスが用意されている）。

次のコードは、500 個の木（それぞれ最大16ノードに制限されている）によるランダムフォレスト分類器を訓練します。


RandomForestClassifier は、少数の例外を除き、木をどのように育てるかを調整するためにDecisionTreeClassifier のハイパーパラメータをすべて持つほか、アンサンブル自体を調整するためにBaggingClassifier のハイパーパラメータもすべて持っています。


ランダムフォレストアルゴリズムは、木を育てるときにさらに無作為性を生み出します。ノードを分
割するときに最良の特徴量を探すのではなく、特徴量の無作為なサブセットから最良の特徴量を探します。その分、木の多様性は増し、それにより（繰り返しになるが）バイアスが上がる分、
分散が下がって、全体としてよりよいモデルが作られます。次のBaggingClassifier は、前のRandomForestClassifier とほぼ同じであります。

In [0]:
bag_clf = BaggingClassifier(
    DecisionTreeClassifier(splitter="random", max_leaf_nodes=16, random_state=42),
    n_estimators=500, max_samples=1.0, bootstrap=True, random_state=42)#n_jobs=-1, random_state=42)

In [0]:
bag_clf.fit(X_train, y_train)
y_pred = bag_clf.predict(X_test)

In [0]:
from sklearn.ensemble import RandomForestClassifier

rnd_clf = RandomForestClassifier(n_estimators=500, max_leaf_nodes=16, random_state=42) # n_jobs=-1, random_state=42)
rnd_clf.fit(X_train, y_train)

y_pred_rf = rnd_clf.predict(X_test)

In [0]:
np.sum(y_pred == y_pred_rf) / len(y_pred)

ランダムフォレストの別の優れた点は、各特徴量の相対的な重要度を簡単に知ることができることであります。scikit-learn は、その特徴量を使うノードが平均して（フォレスト内のすべての木にわ
たり）不純度をどれくらい減らすかを調べることにより、特徴量の重要性を測ります。
scikit-learn は、訓練の後に各特徴量に対してこのスコアを自動的に計算し、その後、すべ
ての重要度の合計が1 になるように結果をスケールします。この結果に得るには、feature_
importances_変数を使用します。たとえば、次のコードは、iris データセットのRandomForestClassifier を訓練し、各機能の重要性を出力します。もっとも重要な特徴量は
花弁の長さ（44%）と幅（42%）で、それと比べるとがく片の長さと幅はそれほど重要ではないようすがわかります（それぞれ11% と2%）。

In [0]:
from sklearn.datasets import load_iris
iris = load_iris()
rnd_clf = RandomForestClassifier(n_estimators=500, n_jobs=-1, random_state=42)
rnd_clf.fit(iris["data"], iris["target"])
for name, score in zip(iris["feature_names"], rnd_clf.feature_importances_):
    print(name, score)

In [0]:
rnd_clf.feature_importances_

In [0]:
plt.figure(figsize=(6, 4))

for i in range(15):
    tree_clf = DecisionTreeClassifier(max_leaf_nodes=16, random_state=42 + i)
    indices_with_replacement = np.random.randint(0, len(X_train), len(X_train))
    tree_clf.fit(X[indices_with_replacement], y[indices_with_replacement])
    plot_decision_boundary(tree_clf, X, y, axes=[-1.5, 2.5, -1, 1.5], alpha=0.02, contour=False)

plt.show()

ランダムフォレストは、特に特徴量選択が必要なときに、どの特徴量が本当の意味で重要かを手っ取り早く調べるために便利であります。

##ブースティング

ブースティング（boosting）は、複数の弱学習器を結合して強学習器を作れるあらゆるアンサンブルメソッドを指します。
ほとんどのブースティングメソッドの一般的な考え方は、逐次的に予測器を訓練し、それによって直前の予測器の修正を試みるというものであります。ブースティングメソッドは多数あるが、人気があるのは、アダブースト（AdaBoost、エイダブーストとも呼ばれています。Adaptive Boosting= 適応的ブースティングの略）と勾配ブースティング（GradientBoosting）である。アダブーストから見ていきましょう。

###アダブースティング

新しい予測器が前の予測器を修正する方法のひとつとしては、前の予測器が過小適合した訓練インスタンスに少しだけ余分に注意を払うというものがあります。そうすると、少しずつ難しい条件につ
いてよく学習した予測器が生まれる。これがアダブーストのテクニックであります。


たとえばアダブースト分類器を作る場合、まずベースの分類器（決定木など）を訓練し、訓練
セットを対象として予測のために使います。そして、分類に失敗した訓練インスタンスの相対的な重み
を上げる。次に、更新された重みを使って第2 の分類器を訓練し、予測に使って、重みを更新す
る。これを繰り返していきます。

下図は、moons データセットを使って連続的に訓練した5 つの予測器の決定境界を示しています（この例では、個々の予測器は、RBF カーネルで高度に正則化されたSVM 分類器である）。
最初の分類器は多くのインスタンスで分類ミスを犯しているので、それらのインスタンスの重みが上げられています。そのため、第2 の分類器は、それらの分類器で性能が上がっています。第5 の分類
器まで、それが連続しています。

右側のグラフは、学習率を半分にしています（つまり、分類ミスを犯したインスタンスに対して各イテレーションが与える重みを半分にしている）。ご覧のように、こ
の逐次的な学習テクニックは、勾配降下法と似ているが、コスト関数を最小化するためにひとつの予測器のパラメータを操作するのではなく、アンサンブルに予測器を追加してアンサンブルを少しずつ改良していきます。

In [0]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.svm import SVC

ada_clf = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=1), n_estimators=200,
    algorithm="SAMME.R", learning_rate=0.5, random_state=42)
ada_clf.fit(X_train, y_train)

In [0]:
plot_decision_boundary(ada_clf, X, y)

In [0]:
m = len(X_train)

plt.figure(figsize=(11, 4))
for subplot, learning_rate in ((121, 1), (122, 0.5)):
    sample_weights = np.ones(m)
    plt.subplot(subplot)
    for i in range(5):
        svm_clf = SVC(kernel="rbf", C=0.05, gamma="auto", random_state=42)
        svm_clf.fit(X_train, y_train, sample_weight=sample_weights)
        y_pred = svm_clf.predict(X_train)
        sample_weights[y_pred != y_train] *= (1 + learning_rate)
        plot_decision_boundary(svm_clf, X, y, alpha=0.2)
        plt.title("learning_rate = {}".format(learning_rate), fontsize=16)
    if subplot == 121:
        plt.text(-0.7, -0.65, "1", fontsize=14)
        plt.text(-0.6, -0.10, "2", fontsize=14)
        plt.text(-0.5,  0.10, "3", fontsize=14)
        plt.text(-0.4,  0.55, "4", fontsize=14)
        plt.text(-0.3,  0.90, "5", fontsize=14)

save_fig("boosting_plot")
plt.show()

###勾配ブースティング

勾配ブースティング（gradient boosting）もよく使われているブース
ティングアルゴリズムであります。勾配ブースティングも、アダブーストと同様に、アンサンブルに前の予測器を改良した予測器を逐次的に加えていきます。しかし、アダブーストのようにイテレー
ションごとにインスタンスの重みを調整するのではなく、新予測器を前の予測器の残差（residual
error）に適合させようとします。


では、ベース予測器として決定木を使った単純な回帰の例を見てみましょう（勾配ブースティングは、
もちろん回帰のタスクでもしっかりと機能する）。これを勾配ブースティング木（gradient tree
boosting）とか勾配ブースティング決定木（GBRT）（gradient boosted regression tree）と呼びます。
まず、訓練セット（たとえば、ノイズのある2 次関数訓練セット）にDecisionTreeRegressorを適合させましょう。

In [0]:
np.random.seed(42)
X = np.random.rand(100, 1) - 0.5
y = 3*X[:, 0]**2 + 0.05 * np.random.randn(100)

In [0]:
from sklearn.tree import DecisionTreeRegressor

tree_reg1 = DecisionTreeRegressor(max_depth=2, random_state=42)
tree_reg1.fit(X, y)

次に、第1 の予測器が作った残差を使って第2 のDecisionTreeRegressor を訓練します。

In [0]:
y2 = y - tree_reg1.predict(X)
tree_reg2 = DecisionTreeRegressor(max_depth=2, random_state=42)
tree_reg2.fit(X, y2)

そして、第2 の予測器が作った残差を使って第3 の回帰器を訓練します。

In [0]:
y3 = y2 - tree_reg2.predict(X)
tree_reg3 = DecisionTreeRegressor(max_depth=2, random_state=42)
tree_reg3.fit(X, y3)

In [0]:
X_new = np.array([[0.8]])

これで3 つの木を含むアンサンブルができた。このアンサンブルは、すべての木の予測を単純に加算するという形で新しいインスタンスの予測をすることができます。

In [0]:
y_pred = sum(tree.predict(X_new) for tree in (tree_reg1, tree_reg2, tree_reg3))

In [0]:
y_pred

In [0]:
def plot_predictions(regressors, X, y, axes, label=None, style="r-", data_style="b.", data_label=None):
    x1 = np.linspace(axes[0], axes[1], 500)
    y_pred = sum(regressor.predict(x1.reshape(-1, 1)) for regressor in regressors)
    plt.plot(X[:, 0], y, data_style, label=data_label)
    plt.plot(x1, y_pred, style, linewidth=2, label=label)
    if label or data_label:
        plt.legend(loc="upper center", fontsize=16)
    plt.axis(axes)

plt.figure(figsize=(11,11))

plt.subplot(321)
plot_predictions([tree_reg1], X, y, axes=[-0.5, 0.5, -0.1, 0.8], label="$h_1(x_1)$", style="g-", data_label="Training set")
plt.ylabel("$y$", fontsize=16, rotation=0)
plt.title("Residuals and tree predictions", fontsize=16)

plt.subplot(322)
plot_predictions([tree_reg1], X, y, axes=[-0.5, 0.5, -0.1, 0.8], label="$h(x_1) = h_1(x_1)$", data_label="Training set")
plt.ylabel("$y$", fontsize=16, rotation=0)
plt.title("Ensemble predictions", fontsize=16)

plt.subplot(323)
plot_predictions([tree_reg2], X, y2, axes=[-0.5, 0.5, -0.5, 0.5], label="$h_2(x_1)$", style="g-", data_style="k+", data_label="Residuals")
plt.ylabel("$y - h_1(x_1)$", fontsize=16)

plt.subplot(324)
plot_predictions([tree_reg1, tree_reg2], X, y, axes=[-0.5, 0.5, -0.1, 0.8], label="$h(x_1) = h_1(x_1) + h_2(x_1)$")
plt.ylabel("$y$", fontsize=16, rotation=0)

plt.subplot(325)
plot_predictions([tree_reg3], X, y3, axes=[-0.5, 0.5, -0.5, 0.5], label="$h_3(x_1)$", style="g-", data_style="k+")
plt.ylabel("$y - h_1(x_1) - h_2(x_1)$", fontsize=16)
plt.xlabel("$x_1$", fontsize=16)

plt.subplot(326)
plot_predictions([tree_reg1, tree_reg2, tree_reg3], X, y, axes=[-0.5, 0.5, -0.1, 0.8], label="$h(x_1) = h_1(x_1) + h_2(x_1) + h_3(x_1)$")
plt.xlabel("$x_1$", fontsize=16)
plt.ylabel("$y$", fontsize=16, rotation=0)

save_fig("gradient_boosting_plot")
plt.show()

上図は、左側にこれら3 つの決定木の予測、右側にアンサンブルの予測を示しています。第1 行
では、アンサンブルに含まれているのは1 つの決定木だけであるため、アンサンブルの予測は第1 の
決定木の予測とまったく同じであります。第2 行では、第1 の決定木の残差を対象として新しい決定
木を訓練しています。アンサンブルの予測は、最初のふたつの決定木の輪だということがわかるはず
です。新たな決定木を追加するたびに、アンサンブルの予測が次第に改善されていくことがわかります。

scikit-learn のGradientBoostingRegressor クラスを使えば、もっと簡単にGBRT アンサンブルを訓練できます。RandomForestRegressor クラスと同様に、このクラスには決定木の
成長を調整するハイパーパラメータ（max_depth、min_samples_leaf など）とアンサンブ
ル訓練を調整するハイパーパラメータ（n_estimators）がある。次のコードは、先ほどと同じアンサンブルを作ります。

In [0]:
from sklearn.ensemble import GradientBoostingRegressor

gbrt = GradientBoostingRegressor(max_depth=2, n_estimators=3, learning_rate=1.0, random_state=42)
gbrt.fit(X, y)

In [0]:
gbrt_slow = GradientBoostingRegressor(max_depth=2, n_estimators=200, learning_rate=0.1, random_state=42)
gbrt_slow.fit(X, y)

In [0]:
plt.figure(figsize=(11,4))

plt.subplot(121)
plot_predictions([gbrt], X, y, axes=[-0.5, 0.5, -0.1, 0.8], label="Ensemble predictions")
plt.title("learning_rate={}, n_estimators={}".format(gbrt.learning_rate, gbrt.n_estimators), fontsize=14)

plt.subplot(122)
plot_predictions([gbrt_slow], X, y, axes=[-0.5, 0.5, -0.1, 0.8])
plt.title("learning_rate={}, n_estimators={}".format(gbrt_slow.learning_rate, gbrt_slow.n_estimators), fontsize=14)

save_fig("gbrt_learning_rate_plot")
plt.show()

learning_rate ハイパーパラメータは、個々の木の影響力を調整します。0.1 などの低い値に設定すると、訓練セットへの適合のためにアンサンブルに多くの決定木を追加しなければならなく
なるが、通常は予測の汎化性能が上がる。これは、収縮（shrinkage）という正則化テクニックで
あります。

上図は、低い学習率で訓練したふたつのGBRT アンサンブルを示しています。左側は決
定木の数が少なすぎて訓練セットに適合できていないのに対し、右側は決定木が多すぎて訓練セッ
トに過学習している。

決定木の最適な数を知るためには、早期打ち切りを使えばよいです。staged_predict() メソッドを使えば、簡単に実装できます。このメソッドは、訓練の各ステージ（決
定木が1 個、決定木が2 個……）でアンサンブルが行った予測を示す反復子を返します。次のコードは、
120 個の決定木でGBRT アンサンブルを訓練し、訓練の各ステージで検証誤差を測定して決定木
の最適な数を調べ、最後にその最適な数の決定木を使って別のGBRT アンサンブルを訓練します。

In [0]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

X_train, X_val, y_train, y_val = train_test_split(X, y, random_state=49)

gbrt = GradientBoostingRegressor(max_depth=2, n_estimators=120, random_state=42)
gbrt.fit(X_train, y_train)

errors = [mean_squared_error(y_val, y_pred)
          for y_pred in gbrt.staged_predict(X_val)]
bst_n_estimators = np.argmin(errors) + 1

gbrt_best = GradientBoostingRegressor(max_depth=2,n_estimators=bst_n_estimators, random_state=42)
gbrt_best.fit(X_train, y_train)

In [0]:
min_error = np.min(errors)

In [0]:
plt.figure(figsize=(11, 4))

plt.subplot(121)
plt.plot(errors, "b.-")
plt.plot([bst_n_estimators, bst_n_estimators], [0, min_error], "k--")
plt.plot([0, 120], [min_error, min_error], "k--")
plt.plot(bst_n_estimators, min_error, "ko")
plt.text(bst_n_estimators, min_error*1.2, "Minimum", ha="center", fontsize=14)
plt.axis([0, 120, 0, 0.01])
plt.xlabel("Number of trees")
plt.title("Validation error", fontsize=14)

plt.subplot(122)
plot_predictions([gbrt_best], X, y, axes=[-0.5, 0.5, -0.1, 0.8])
plt.title("Best model (%d trees)" % bst_n_estimators, fontsize=14)

save_fig("early_stopping_gbrt_plot")
plt.show()

実際に訓練を早い段階で打ち切って早期打ち切りを実現することもできます（最初に大量の決定木
を訓練してから、最適な個数を探すのではなく）。そのためには、warm_start=True を設定し
て、fit() メソッドを呼び出したときにscikit-learn が既存の決定木を残して、漸進的に訓練を実
行できるようにします。次のコードは、検証誤差が5 回連続で改善されないときに訓練を打ち切ります。

In [0]:
gbrt = GradientBoostingRegressor(max_depth=2, warm_start=True, random_state=42)

min_val_error = float("inf")
error_going_up = 0
for n_estimators in range(1, 120):
    gbrt.n_estimators = n_estimators
    gbrt.fit(X_train, y_train)
    y_pred = gbrt.predict(X_val)
    val_error = mean_squared_error(y_val, y_pred)
    if val_error < min_val_error:
        min_val_error = val_error
        error_going_up = 0
    else:
        error_going_up += 1
        if error_going_up == 5:
            break  # early stopping

In [0]:
print(gbrt.n_estimators)

In [0]:
print("Minimum validation MSE:", min_val_error)

GradientBoostingRegressor クラスは、個々の決定木を訓練するために使われる訓練イ
ンスタンスの割合を指定するsubsample ハイパーパラメータもサポートしています。たとえば、
subsample=0.25 とすると、個々の決定木は無作為に選択された25% の訓練インスタンスを
使って訓練される。これはバイアスを少し上げた分、分散を
下げます。また、訓練のスピードもかなり上がります。このテクニックを確率的勾配ブースティング
（stochastic gradient boosting）と呼びます。